In [433]:
import numpy as np
import random
from pprint import pprint

In [434]:
class Cardan:
    def __init__(self, n: int = 4):
        self.n = n
        self.grid = None
        self.mask = None
        self.cipher = None

    def create_grid(self, shuffle: bool = False):
        quarter_size = self.n // 2
        quarter = np.arange(1, quarter_size**2+1).reshape((quarter_size, quarter_size))
        if shuffle:
            np.random.shuffle(quarter)
        up_left_quarter = quarter
        up_right_quarter = np.rot90(up_left_quarter, k=-1)
        down_right_quarter = np.rot90(up_right_quarter, k=-1)
        down_left_quarter = np.rot90(down_right_quarter, k=-1)

        up_half = np.concatenate((up_left_quarter, up_right_quarter), axis=1)
        down_half = np.concatenate((down_left_quarter, down_right_quarter), axis=1)

        self.grid = np.concatenate((up_half, down_half), axis=0)
        return self.grid

    def create_mask(self):
        stat = {}
        for i in range(self.n):
            for j in range(self.n):
                stat[self.grid[i,j]] = stat.get(self.grid[i,j], []) + [(i,j)]

        self.mask = np.zeros((self.n, self.n))
        for fields in stat.values():
            i, j = random.choice(fields)
            self.mask[i,j] = 1
        return self.mask
    
    def create_cipher(self, text: str, verbose: bool = False):
        text += "_" * (self.n**2 - len(text))
        text_i = 0
        mask = self.mask.copy()
        self.cipher = [["_" for _ in range(self.n)] for _ in range(self.n)]
        for _ in range(4):
            if verbose:
                pprint(self.cipher)
                print("="*100)
                pprint(mask)
            for i in range(self.n):
                for j in range(self.n):
                    if mask[i,j]:
                        self.cipher[i][j] = text[text_i]
                        text_i += 1
            mask = np.rot90(mask, k=-1)
        return self.cipher
    
    def get_cryptogram(self):
        return "".join(["".join(row) for row in self.cipher])
    
    def get_cryptogram_key(self):
        return "".join(["".join([str(int(i)) for i in row]) for row in self.mask])
    
    def set_cryptogram(self, cryptogram: str) -> list[list[str]]:
        self.n = int(len(cryptogram)**0.5)
        self.cipher = [
            list(cryptogram[self.n*i:self.n*i+self.n]) for i in range(self.n)
        ]
        return self.cipher
    
    def set_cryptogram_key(self, cryptogram_key: str) -> np.ndarray:
        self.n = int(len(cryptogram_key)**0.5)
        self.mask = np.array([
            [int(sym) for sym in cryptogram_key[self.n*i:self.n*i+self.n]] for i in range(self.n)
        ])

        return self.mask
    
    def create_mask_from_variant(self, variant: dict[int, tuple[int,int]]) -> np.ndarray:
        if len(variant) != (self.n//2)**2:
            raise ArithmeticError(f"bad digits for creating mask {len(variant)=}, {((self.n//2)**2)=}, {self.n}")
        self.mask = np.zeros((self.n, self.n))
        for position in variant.values():
            self.mask[position[0], position[1]] = 1
        return self.mask
                
    def decode_cipher(self, verbose: bool = False) -> str:
        result = ""
        mask = self.mask.copy()
        for _ in range(4):
            if verbose:
                pprint(self.cipher)
                print(result[len(result)-self.n:len(result)])
                print(result)
                print("="*100)
                pprint(mask)
            for i in range(self.n):
                for j in range(self.n):
                    if mask[i,j]:
                        result += self.cipher[i][j]
            mask = np.rot90(mask, k=-1)
        return result

        

In [435]:
def rec_cryptogram(cardan: Cardan, first_substring: str, variant: dict[int, tuple[int,int]], last_find_position: tuple[int,int] = (0,-1)):
    if len(variant)*4 == cardan.n**2 or len(first_substring) == len(variant):
        return [variant]
    variants = []
    letter = first_substring[len(variant)]
    for i in range(last_find_position[0], cardan.n):
        for j in range((i==last_find_position[0])*(last_find_position[1]+1), cardan.n):
            if cardan.cipher[i][j] == letter:
                digit = cardan.grid[i,j]
                new_variant = variant.copy()
                new_variant[int(digit)] = (i,j)
                res_rec = rec_cryptogram(cardan, first_substring, variant=new_variant, last_find_position=(i,j))
                if res_rec:
                    variants += res_rec
    
    return variants

def compute_all_variants(cardan: Cardan, bad_variant: dict[int, tuple[int,int]]):
    block = (cardan.n//2)**2
    variants = [bad_variant]
    for digit in range(1, block+1):
        if bad_variant.get(digit):
            continue
        new_coords = []
        for i in range(cardan.n):
            for j in range(cardan.n):
                if cardan.grid[i,j] == digit:
                    new_coords.append((i,j))
        new_variants = []
        for variant in variants:
            for new_coord in new_coords:
                new_variant = variant.copy()
                new_variant[digit] = new_coord
                new_variants.append(new_variant)
        variants = new_variants
    return variants


def brutforce(cryptogram: str, first_substring: str, verbose: bool = False) -> list[str]:
    results = []
    cardan = Cardan()
    cardan.set_cryptogram(cryptogram)
    cardan.create_grid()

    if (cardan.n//2)**2 > len(first_substring):
        print(f"будет многовато вариантов, желательно длину хотя бы {((cardan.n//2)**2)=}")
    
    if (cardan.n//2)**2 < len(first_substring):
        first_substring = first_substring[:((cardan.n//2)**2)]

    variants = rec_cryptogram(cardan, first_substring, variant={})
    new_variants = []
    for variant in variants:
        if len(variant) < (cardan.n//2)**2:
            new_variants += compute_all_variants(cardan, variant)
        elif len(variant) == (cardan.n//2)**2:
            new_variants.append(variant)
    variants = new_variants
    for variant in variants:
        cardan.create_mask_from_variant(variant)
        result = cardan.decode_cipher()
        if verbose:
            print(cardan.mask)
            print(variant)
        results.append(result)
    return results
    

In [436]:
cardan = Cardan(n=6)

In [437]:
cardan.create_grid()

array([[1, 2, 3, 7, 4, 1],
       [4, 5, 6, 8, 5, 2],
       [7, 8, 9, 9, 6, 3],
       [3, 6, 9, 9, 8, 7],
       [2, 5, 8, 6, 5, 4],
       [1, 4, 7, 3, 2, 1]])

In [438]:
cardan.create_mask()

array([[1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [439]:
cardan.create_cipher("привет, но без пока", verbose=True)

[['_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_']]
array([[1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])
[['п', '_', '_', '_', 'р', '_'],
 ['_', 'и', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_'],
 ['в', 'е', 'т', '_', ',', ' '],
 ['_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', 'н', '_']]
array([[0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.]])
[['п', '_', 'о', '_', 'р', ' '],
 ['_', 'и', 'б', '_', 'е', '_'],
 ['_', '_', 'з', '_', '_', '_'],
 ['в', 'е', 'т', '_', ',', ' '],
 [' ', '_', 'п', '_', '_', 'о'],
 ['_', '_', 'к', '_', 'н', '_']]
array([[

[['п', 'а', 'о', '_', 'р', ' '],
 ['_', 'и', 'б', '_', 'е', '_'],
 ['_', '_', 'з', '_', '_', '_'],
 ['в', 'е', 'т', '_', ',', ' '],
 [' ', '_', 'п', '_', '_', 'о'],
 ['_', '_', 'к', '_', 'н', '_']]

In [440]:
cardan.get_cryptogram()

'пао_р _иб_е___з___вет_,  _п__о__к_н_'

In [441]:
cardan.get_cryptogram_key()

'100010010000000000111011000000000010'

In [442]:
cardan.set_cryptogram("п_р_ио_в__ еа_б_те__,_ н__з _п__о_к_")

[['п', '_', 'р', '_', 'и', 'о'],
 ['_', 'в', '_', '_', ' ', 'е'],
 ['а', '_', 'б', '_', 'т', 'е'],
 ['_', '_', ',', '_', ' ', 'н'],
 ['_', '_', 'з', ' ', '_', 'п'],
 ['_', '_', 'о', '_', 'к', '_']]

In [443]:
cardan.set_cryptogram_key("101010010001000010001011000000000000")

array([[1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [444]:
cardan.decode_cipher(verbose=True)

[['п', '_', 'р', '_', 'и', 'о'],
 ['_', 'в', '_', '_', ' ', 'е'],
 ['а', '_', 'б', '_', 'т', 'е'],
 ['_', '_', ',', '_', ' ', 'н'],
 ['_', '_', 'з', ' ', '_', 'п'],
 ['_', '_', 'о', '_', 'к', '_']]


array([[1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])
[['п', '_', 'р', '_', 'и', 'о'],
 ['_', 'в', '_', '_', ' ', 'е'],
 ['а', '_', 'б', '_', 'т', 'е'],
 ['_', '_', ',', '_', ' ', 'н'],
 ['_', '_', 'з', ' ', '_', 'п'],
 ['_', '_', 'о', '_', 'к', '_']]
вет, н
привет, н
array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1],
       [0, 0, 1, 0, 1, 0]])
[['п', '_', 'р', '_', 'и', 'о'],
 ['_', 'в', '_', '_', ' ', 'е'],
 ['а', '_', 'б', '_', 'т', 'е'],
 ['_', '_', ',', '_', ' ', 'н'],
 ['_', '_', 'з', ' ', '_', 'п'],
 ['_', '_', 'о', '_', 'к', '_']]
ез пок
привет, но без пок
array([[0, 0, 0, 0, 0, 0],
       [

'привет, но без пока_________________'

In [445]:
results = brutforce(
    cryptogram="о___ ап____р_бе__и_зв__е_ т, _н_пок_",
    first_substring="привет,",
    verbose=False
)

будет многовато вариантов, желательно длину хотя бы ((cardan.n//2)**2)=9


In [446]:
results

['оп_ривет, а_безпок______ ________ н_',
 'оп_ривет, абез пок______ _________н_',
 'оприве т, а_безпок_______________ н_',
 'опривет,  абез пок________________н_',
 'ап_ривет, _безпок_______ _но______ _',
 'ап_ривет, без пок_______ _но________',
 'априве т, _безпок_________но______ _',
 'апривет,  без пок_________но________',
 'п_ривет,но _безпока______ _______ __',
 'п_ривет,но без пока______ __________',
 'приве т,но _безпока______________ __',
 'привет, но без пока_________________',
 'п_ривет,_ _безнпоко______ ____а___ _',
 'п_ривет,_ без нпоко______ ____а_____',
 'приве т,_ _безнпоко___________а___ _',
 'привет, _ без нпоко___________а_____']